<a href="https://colab.research.google.com/github/davidfague/Stylized-ReducedOrder-L5-Model/blob/main/Segment_Axial_Currents_11_3_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as ss
from mpl_toolkits import mplot3d

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Stylized-ReducedOrder-L5-Model/

Mounted at /content/drive
/content/drive/MyDrive/Stylized-ReducedOrder-L5-Model


In [3]:
output_folder = 'Detailedoutput_control'
v = h5py.File('./{}/Detailedv_report.h5'.format(output_folder),'r') #read segment voltage traces
segs = pd.read_csv('Detailedoutput_control/DetailedSegments.csv') # read segment values

f = h5py.File('./{}/ecp.h5'.format(output_folder),'r') #read ecp testing another project

OSError: ignored

In [ ]:
f.visit(print)
f.name
# data_raw = f['ecp/data'][()]

In [ ]:
#Usefull subsetting method
# print(segs[(segs.Type=='soma')]) #how to separate by type
somasegIDs = segs[(segs.Type=='soma')]['SEG ID'] #how to pull a column after separating by type

In [ ]:
segs.head() # visualize categories

In [ ]:
segIDs=segs['SEG ID']

# segs = segs.set_index(['Type','Sec ID']).join(segs_degrees.set_index(['Type','Sec ID'])).reset_index() #not sure what this does
segs['Sec ID'] = segs['Sec ID'].astype(int)
segs['X'] = segs['X'].astype(float)
segs['Elec_distanceQ'] = 'None'
AllSegXCoord=segs['Coord X']
AllSegYCoord=segs['Coord Y']
AllSegZCoord=segs['Coord Z']
AllSegSec_ID=segs['Sec ID']
AllSegSec_name=segs['Type']
segLs=segs["Seg_L"]
segxs=segs["X"]
segdists=segs["Distance"]
SegSecL=segs["Section_L"]
segdiams=segs["Seg_diam"]
SegSecnseg=segs["Section_nseg"]
secRas=segs["Section_Ra"]
psegIDs=segs["ParentSegID"]
secids=segs["Sec ID"]
bmtkid=segs["BMTK ID"]

psegids=[]
for i in range(len(psegIDs)):
  if np.isnan(psegIDs[i]) == False:
    psegids.append(int(psegIDs[i]))
  else:
    psegids.append(psegIDs[i])

In [ ]:
print(psegids)

In [ ]:
class childseg():
  def __init__(self,seg,sectionindex,sectionname,color,XCoord,YCoord,ZCoord,secRa,segdiam,segx,segdist,sectionL,sectionNseg,segL,seg_v):
    self.seg=seg
    self.sectionindex=sectionindex
    self.sectionname=sectionname
    self.color=color
    self.XCoord=XCoord
    self.YCoord=YCoord
    self.ZCoord=ZCoord
    self.secRa=secRa
    self.segdiam=segdiam
    self.x=segx
    self.segdist=segdist
    self.secL=sectionL
    self.secnseg=sectionNseg
    self.segL=segL
    self.seg_v=seg_v
    self.name='sec:'+str(self.sectionname)+str(self.sectionindex)+' seg:'+str(self.seg)

In [ ]:
print(len(segIDs)) # number of segments

In [ ]:
seginterest=[1685,1686,1900] #choose segments indexes to probe
colors=['blue','purple','orange'] #specify colorchoose colors

In [ ]:
probes=[]
j=0 #index color list
class probe():
  def __init__(self,seg,sectionindex,sectionname,color,XCoord,YCoord,ZCoord,secRa,segdiam,segx,segdist,sectionL,sectionNseg,segL,seg_v,parentseg):
    self.seg=seg
    self.sectionindex=sectionindex
    self.sectionname=sectionname
    self.color=color
    self.XCoord=XCoord
    self.YCoord=YCoord
    self.ZCoord=ZCoord
    self.secRa=secRa
    self.segdiam=segdiam
    self.x=segx
    self.segdist=segdist
    self.secL=sectionL
    self.secnseg=sectionNseg
    self.segL=segL
    self.seg_v=seg_v
    self.name='sec:'+str(self.sectionname)+str(self.sectionindex)+' seg:'+str(self.seg)
    self.adjprobes=[]
    self.parentprobe=[]
    self.childprobes=[]
    self.axialcurrents=[]
    self.parentaxialcurrent=[]
    self.childrenaxialcurrents=[]
    self.halfsegRa=.01*self.secRa*(self.secL/2/self.secnseg)/(np.pi*(self.segdiam/2)**2)
    self.parentseg=parentseg

    # try:
    #   self.axial_current=[Record_Axial_Current(section,single_seg=True)]
    # except:
    #   print(section)
for i in range(len(segIDs)):
  try:
    newprobe=probe(seg=i,sectionindex=AllSegSec_ID[i],sectionname=str(AllSegSec_name[i]),color=colors[j],XCoord=AllSegXCoord[i],YCoord=AllSegYCoord[i],ZCoord=AllSegZCoord[i],
                  secRa=secRas[i],segdiam=segdiams[i],segx=segxs[i],segdist=segdists[i],sectionL=SegSecL[i],sectionNseg=SegSecnseg[i],segL=segLs[i],
                  seg_v=v['report']['biophysical']['data'][:,i],parentseg=psegids[i])
  except: #use red if colors isn't long enough
        newprobe=probe(seg=i,sectionindex=AllSegSec_ID[i],sectionname=str(AllSegSec_name[i]),color='r',XCoord=AllSegXCoord[i],YCoord=AllSegYCoord[i],ZCoord=AllSegZCoord[i],
                secRa=secRas[i],segdiam=segdiams[i],segx=segxs[i],segdist=segdists[i],sectionL=SegSecL[i],sectionNseg=SegSecnseg[i],segL=segLs[i],
                seg_v=v['report']['biophysical']['data'][:,i],parentseg=psegids[i])
  j+=1 #index color list
  probes.append(newprobe)

In [ ]:
print(psegids)

In [ ]:
# psegs=[] #list of segment ids for a section's parentseg(), which is the segment that the section is attached to
# parentsegs=list(set(psegids)) # remove duplicates of the list of segment parentsegmentIDs
# # print(parentsegs)
# for j in range(len(segIDs)):
#   for i in range(len(parentsegs)):
#     if np.isnan(parentsegs[i]) == False: #remove nan
#       psegs.append(int(parentsegs[i])) #convert to integer again
#     # else: #if nan, then this segment is really the first segment
#     psegs.append(int())
# # print('psegs:',psegs)

firstsegs = [] #list of segment indices for each section's first segment
sections=[] #list of already seen section ids

for i in range(len(bmtkid)):
    if bmtkid[i] not in sections: #make sure secid is unique
      sections.append(bmtkid[i]) #add unique secid to list of already seen secids
      firstsegs.append(segIDs[i]) #get a list of first segment by unique section id

print('firstsegs:',firstsegs)

Below code is being adjusted to only probe parentsegments, and 

In [ ]:
#getting adjacent probes using parent segmemt IDs
for i in range(len(psegids)): #iterate through segment index
  if np.isnan(psegids[i]) == False:
    for seg in segIDs: #check segIDs
      if psegids[i]==seg: #find parent seg from parent seg id
        probes[seg].adjprobes.append(probes[i]) #add child probe to probe's adjprobe list
        probes[seg].childprobes.append(probes[i]) #add child probe to probe's childprobe list

In [ ]:
for i in range(len(segIDs)):
  if np.isnan(psegids[i]) == False:
    probes[i].adjprobes.append(probes[int(psegids[i])]) #add parent seg probe to adj probes
    probes[i].parentprobe.append(probes[int(psegids[i])]) #add parent seg probe to parent probes

In [ ]:
#compute axial currents between probes
for probe in probes:
  for adjprobe in probe.adjprobes:
    axc=(probe.seg_v-adjprobe.seg_v)/(probe.halfsegRa+adjprobe.halfsegRa) #compute axial current using (v_in-v_out)/(halfsegRa+halfsegRa)
    probe.axialcurrents.append(axc)
    if [adjprobe] == probe.parentprobe:
      probe.parentaxialcurrent.append(axc)
    elif adjprobe in probe.childprobes:
      probe.childrenaxialcurrents.append(axc)

In [ ]:
for i in range(len(probes)):
  print(probes[i].adjprobes)

In [ ]:
s=807
print(probes[s].parentprobe)
print(probes[s].adjprobes)
print(probes[s].childprobes)
print(probes[s].parentaxialcurrent)
print(probes[s].axialcurrents)
print(probes[s].childrenaxialcurrents)

In [ ]:
print(len(probes))
print(len(segIDs))
print(len(psegids))

In [ ]:
#check adjprobes lists
# for i in range(len(probes)):
#   print(probes[i].adjprobes)

In [ ]:
elev=90 
azim=-90 #20
figsize=(10,12)
# apicsegs=segs[(segs.Type=='apic')]['segmentID']
#create 3d axes
fig = plt.figure(figsize=figsize)
ax = plt.axes(projection='3d')
ax.plot3D(AllSegXCoord, AllSegYCoord,AllSegZCoord,'.',color='k',markersize=7)
# for i in range(len(AllSegXCoord)):
#   ax.scatter(AllSegXCoord[i], AllSegYCoord[i],AllSegZCoord[i],'.',color='k')
# import pdb;pdb.set_trace()
print(probes[0].color)
# for i in range(len(probes)):
ax.scatter(xs=AllSegXCoord[probes[firstsegs[-61]].parentprobe[0].seg],
        ys=AllSegYCoord[probes[firstsegs[-61]].parentprobe[0].seg],
        zs=AllSegZCoord[probes[firstsegs[-61]].parentprobe[0].seg],
        marker='*',color=probes[probes[firstsegs[-61]].parentprobe[0].seg].color)
plt.savefig('Probe Locations.png')
# ax.auto_scale_xyz(*box.T)
ax.view_init(elev,azim)
plt.show()

In [ ]:
plt.figure(figsize=(3,10))
ax = plt.plot(AllSegXCoord, AllSegYCoord,'.',color='k',markersize=6)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
plt.box(False)

for i in range(len(probes)):
  plt.plot(AllSegXCoord[probes[i].seg],
         AllSegYCoord[probes[i].seg],
         '*',color=probes[i].color)
  for j in range(len(probes[i].adjprobes)):
      plt.plot(AllSegXCoord[probes[i].adjprobes[j].seg],
          AllSegYCoord[probes[i].adjprobes[j].seg],
          '*',color='blue')
# for i in range(len(probes)):
#   plt.plot(AllSegXCoord[probes[i].seg],
#          AllSegYCoord[probes[i].seg],
#          '*',color=probes[i].color)
plt.savefig('Probe Locations.png')

In [ ]:
plt.figure(figsize=(3,10))
ax = plt.plot(AllSegXCoord, AllSegYCoord,'.',color='k',markersize=6)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
plt.box(False)

for i in firstsegs:
  plt.plot(AllSegXCoord[probes[i].seg],
         AllSegYCoord[probes[i].seg],
         '*',color=probes[i].color)
  # for j in range(len(probes[i].adjprobes)):
  #     plt.plot(AllSegXCoord[probes[i].adjprobes[j].seg],
  #         AllSegYCoord[probes[i].adjprobes[j].seg],
  #         '*',color='blue')

nexus=[probes[firstsegs[-51]].parentprobe[0].seg,probes[firstsegs[-61]].parentprobe[0].seg] #check parentseg()'s
# adjp=[0] #indexes childprobes of nexus probes to find the adjacent
for i in nexus:
  plt.plot(AllSegXCoord[probes[i].seg],
         AllSegYCoord[probes[i].seg],
         '*',color='m')
  # see all adj probes
  # for j in range(len(probes[].adjprobes)):
  #   plt.plot(AllSegXCoord[probes[].adjprobes[j].seg],
  #       AllSegYCoord[probes[psegs[i]].adjprobes[j].seg],
  #       '*',color='blue')
#see child probes
  for j in range(len(probes[i].childprobes)):
      plt.plot(AllSegXCoord[probes[i].childprobes[j].seg],
          AllSegYCoord[probes[i].childprobes[j].seg],
          '*',color='orange')
#see parent probes      
  for j in range(len(probes[i].parentprobe)):
      plt.plot(AllSegXCoord[probes[i].parentprobe[j].seg],
          AllSegYCoord[probes[i].parentprobe[j].seg],
          '*',color='blue')

In [ ]:
probes[firstsegs[-61]].parentprobe[0].seg

In [ ]:
# print(psegids[1474])

In [ ]:
nexusandsoma=[0,int(psegids[int(psegids[1700])])]
plt.figure(figsize=(3,10))
ax = plt.plot(AllSegXCoord, AllSegYCoord,'.',color='k',markersize=6)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
plt.box(False)

for i in nexusandsoma:
  plt.plot(AllSegXCoord[probes[i].seg],
         AllSegYCoord[probes[i].seg],
         '*',color=probes[i].color)
  for j in range(len(probes[i].adjprobes)):
      plt.plot(AllSegXCoord[probes[i].adjprobes[j].seg],
          AllSegYCoord[probes[i].adjprobes[j].seg],
          '*',color='blue')
      
for i in nexusandsoma:
  print('seg:',probes[i].seg,'adjprobes:',len(probes[i].adjprobes))

In [ ]:
for i in range(len(probes[nexusandsoma[1]].childprobes)):
  print(probes[nexusandsoma[1]].childprobes[i].seg)

In [ ]:
M=10
for i in range(M):
  print(psegids[nexusandsoma[1]-M+i:nexusandsoma[1]-M+i+1]) #print previous M values
print('nexusseg is:',nexusandsoma[1]) # indicate the segment
for i in range(M):
  print(psegids[nexusandsoma[1]+i:nexusandsoma[1]+i+1]) # print next M values

print('***')
print(psegids[nexusandsoma[1]-10:nexusandsoma[1]+10])

Try to adjust Probes indices until they are adjacent, at key points.

In [ ]:
seg_v=v['report']['biophysical']['data'][:,0] # get segment 0 voltage trace
print(len(seg_v)) #should be equivalnt to (tstop=1000ms)/(dt=0.1ms)=10000

In [ ]:
# class axc():
#   def __init__(self,name,i,color):
#     self.name=name
#     self.current=i
#     self.color=color

# axialcurrents=[]
# parentprobes=[]
# parentprobe=probes[0]  #parent segment probe #blue
# adjprobes=[probes[1],probes[2]] # array that should denote probes for child segments of the parent segment probe #purple, orange

# parentaxr=.01*parentprobe.secRa*(parentprobe.secL/2/parentprobe.secnseg)/(np.pi*(parentprobe.segdiam/2)**2) #compute half segment axial resistance
# for adjprobe in adjprobes:
#   adjprobeaxr=.01*adjprobe.secRa*(adjprobe.secL/2/adjprobe.secnseg)/(np.pi*(adjprobe.segdiam/2)**2) #compute half segment axial resistance
#   axialcurrent=(parentprobe.seg_v-adjprobe.seg_v)/(parentaxr+adjprobeaxr) #compute axial current using (v_in-v_out)/(halfsegRa+halfsegRa)
#   # axial_current=axc(name=adjprobe.name,i=axialcurrent,color=adjprobe.color)
#   # axialcurrents.append(axial_current)
#   adjprobe.axialcurrent=axialcurrent

# parentprobe.adjprobes=adjprobes
# parentprobe.axialcurrents=axialcurrents
# parentprobes.append(parentprobe)

In [ ]:
tstop=1000
dt=0.1
t=np.arange(0,tstop,dt)
def get_probe(probe):
  name=probe.name
  segment=probe.seg
  color=probe.color
  axialcurrents=probe.axialcurrents
  childrenaxialcurrents=probe.childrenaxialcurrents
  parentaxialcurrent=probe.parentaxialcurrent
  adjprobes=probe.adjprobes
  parentprobe=probe.parentprobe
  childprobes=probe.childprobes
  return name,segment,color,axialcurrents,childrenaxialcurrents,parentaxialcurrent,adjprobes,parentprobe,childprobes

def plot_axial_current(probe):
  name,segment,color,axialcurrents,childrenaxialcurrents,parentaxialcurrent,adjprobes,parentprobe,childprobes=get_probe(probe)
  title=name+' Axial Current'
  plt.figure(figsize=(12.8, 4.8))
  # for i,AC in enumerate(axial_current):
  for c in range(len(parentaxialcurrent)):
      # plt.plot(t,ac[dend_type].ravel(),label=dend_type)
      plt.plot(t,parentaxialcurrent[c],label=parentprobe[c].name,color='purple')
  for c in range(len(childrenaxialcurrents)):
      plt.plot(t,childrenaxialcurrents[c],label=childprobes[c].name,color='cyan')
  plt.ylabel('nA')
  plt.legend()
  plt.title(title)
  plt.xlabel('time (ms)')
  plt.show()

def plot_sum_child_axial_current(probe):
  name,segment,color,axialcurrents,childrenaxialcurrents,parentaxialcurrent,adjprobes,parentprobe,childprobes=get_probe(probe)
  title=name+' Axial Current'
  plt.figure(figsize=(12.8, 4.8))
  # for i,AC in enumerate(axial_current):
  for c in range(len(parentaxialcurrent)):
      # plt.plot(t,ac[dend_type].ravel(),label=dend_type)
      plt.plot(t,parentaxialcurrent[c],label=parentprobe[c].name,color='purple')
  axc=np.zeros(len(childrenaxialcurrents[0]))
  for c in range(len(childrenaxialcurrents)):
    axc+=childrenaxialcurrents[c] #sum the children axial currents
  for c in range(len(childrenaxialcurrents)):
      plt.plot(t,axc,label=childprobes[c].name,color='cyan')
  plt.ylabel('nA')
  plt.legend()
  plt.title(title)
  plt.xlabel('time (ms)')
  plt.show()

def sumcurrents(currents):
  current=np.zeros(len(currents[0]))
  for i in range(len(currents)):
    current+=currents[i]
  return current

def sumcurrentsfromprobes(probes):
  probesaxc=np.zeros(len(probes[0].axialcurrents[0]))
  for i in range(len(probes)):
    axc=sumcurrents(probes[i].axialcurrents)
    probesaxc+=axc
  return probesaxc
  

def getsummedprobenames(probe,arg):
  name=''
  if arg == 'adjprobes':
    for i in range(len(probe.adjprobes)):
      name+=(' '+probe.adjprobes[i].name)
    print(name)
  elif arg == 'childprobes':
    for i in range(len(probe.childprobes)):
      name+=(' '+probe.childprobes[i].name)
    print(name)
  elif arg == 'parentprobe':
    for i in range(len(probe.parentprobe)):
      name+=(' '+probe.parentprobe[i].name)
  return name


def plot_net_axial_current(probe,title=None):
  name,segment,color,axialcurrents,childrenaxialcurrents,parentaxialcurrent,adjprobes,parentprobe,childprobes=get_probe(probe)
  if title==None:
    title=name+' Net Axial Current'
  else:
    title=title+' Net Axial Current'
  plt.figure(figsize=(12.8, 4.8))
  # for i,AC in enumerate(axial_current):
  # for c in range(len(parentaxialcurrent)):
  #     # plt.plot(t,ac[dend_type].ravel(),label=dend_type)
  #     plt.plot(t,parentaxialcurrent[c],label=parentprobe[c].name,color='purple')
  axc=sumcurrents(axialcurrents)
  label=getsummedprobenames(probe,'adjprobes')
  plt.plot(t,axc,label=label,color='red')

  plt.ylabel('nA')
  plt.legend()
  plt.title(title)
  plt.xlabel('time (ms)')
  plt.show()

def plot_net_axial_currentmultiprobe(probes,title=None):
  # name,segment,color,axialcurrents,childrenaxialcurrents,parentaxialcurrent,adjprobes,parentprobe,childprobes=get_probe(probe)
  if title==None:
    title=name+' Net Axial Current'
  else:
    title=title+' Net Axial Current'
  plt.figure(figsize=(12.8, 4.8))
  # for i,AC in enumerate(axial_current):
  # for c in range(len(parentaxialcurrent)):
  #     # plt.plot(t,ac[dend_type].ravel(),label=dend_type)
  #     plt.plot(t,parentaxialcurrent[c],label=parentprobe[c].name,color='purple')
  axc=sumcurrentsfromprobes(probes)
  label=''
  # for i in range(len(probes)):
  #   label+=getsummedprobenames(probes[i],'adjprobes')
  plt.plot(t,axc,label=label,color='red')

  plt.ylabel('nA')
  plt.legend()
  plt.title(title)
  plt.xlabel('time (ms)')
  plt.show()

In [ ]:
nexus=[probes[probes[firstsegs[-61]].parentprobe[0].seg]] #select nexus probe
soma=[]
for i in somasegIDs:
  soma.append(probes[i])
# soma=[probes[0:5]] #select soma probes
print(soma)
print(nexus)

In [ ]:
for probe in nexus:
  # import pdb; pdb.set_trace()
  plot_net_axial_current(probe, 'Nexus')

In [ ]:
for probe in [soma]:
  # import pdb; pdb.set_trace()
  plot_net_axial_currentmultiprobe(probes, 'Soma')